## Forward and inverse models for Experiment 2

In [ ]:
import h5py

with h5py.File('./data/magnetic/kuka_dir-rnd_1500k_n10_v2_f10.h5', 'r') as hf:
    kuka_fwd = hf['kuka_magnetic_fwd'][()]
    kuka_inv = hf['kuka_magnetic_inv'][()]

In [ ]:
# assert len(obj_xyz_0) == 3
# assert len(obj_rot_0) == 4
# assert len(color) == 3
# assert len(joints_0) == n_joints
# assert len(eff_xyz_0) == 3
# assert len(eff_rot_0) == 4

import numpy as np
from sklearn.preprocessing import normalize

X_fwd, y_fwd = np.hsplit(kuka_fwd, [33])

(
    X_fwd_obj_xyz, X_fwd_obj_rot, X_fwd_color,
    X_fwd_joints, X_fwd_eff_xyz, X_fwd_eff_rot, X_fwd_magnet,
    X_fwd_action
) = np.hsplit(X_fwd, [3, 7, 10, 17, 20, 24, 25])

X_fwd_obj_rot_axis = normalize(X_fwd_obj_rot[:, :3], norm='l2')
X_fwd_eff_rot_axis = normalize(X_fwd_eff_rot[:, :3], norm='l2')

X_fwd_axis = np.hstack([
    X_fwd_obj_xyz, X_fwd_obj_rot_axis, X_fwd_color,
    X_fwd_joints, X_fwd_eff_xyz, X_fwd_eff_rot_axis, X_fwd_magnet,
    X_fwd_action
])

(
    y_fwd_obj_xyz, y_fwd_obj_rot, y_fwd_color,
    y_fwd_joints, y_fwd_eff_xyz, y_fwd_eff_rot, y_fwd_magnet
) = np.hsplit(y_fwd, [3, 7, 10, 17, 20, 24])

y_fwd_obj_rot_axis = normalize(y_fwd_obj_rot[:, :3], norm='l2')
y_fwd_eff_rot_axis = normalize(y_fwd_eff_rot[:, :3], norm='l2')

X_inv, y_inv = np.hsplit(kuka_inv, [50])
(
    # S0
    X_inv_obj_xyz_0, X_inv_obj_rot_0, X_inv_color_0,
    X_inv_joints_0, X_inv_eff_xyz_0, X_inv_eff_rot_0, X_inv_magnet_0,

    # S1
    X_inv_obj_xyz_1, X_inv_obj_rot_1, X_inv_color_1,
    X_inv_joints_1, X_inv_eff_xyz_1, X_inv_eff_rot_1, X_inv_magnet_1
) = np.hsplit(
    X_inv,
    [
        3, 7, 10, 17, 20, 24, 25,  # S0 
        28, 32, 35, 42, 45, 49  # S1
    ]
)

X_inv_obj_rot_0_axis = normalize(X_inv_obj_rot_0[:, :3], norm='l2')
X_inv_eff_rot_0_axis = normalize(X_inv_eff_rot_0[:, :3], norm='l2')
X_inv_obj_rot_1_axis = normalize(X_inv_obj_rot_1[:, :3], norm='l2')
X_inv_eff_rot_1_axis = normalize(X_inv_eff_rot_1[:, :3], norm='l2')

X_inv_reduced = np.hstack([
    X_inv_obj_xyz_0, X_inv_obj_rot_0_axis, X_inv_color_0,
    X_inv_joints_0, X_inv_eff_xyz_0, X_inv_eff_rot_0_axis, X_inv_magnet_0,

    X_inv_obj_xyz_1, X_inv_obj_rot_1_axis, X_inv_color_1,
    X_inv_eff_xyz_1, X_inv_eff_rot_1_axis, X_inv_magnet_1
])

y_inv_joints, y_inv_magnet = np.hsplit(y_inv, [7])

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

inp = Input(shape=(31,))
h = Dense(50, activation='tanh')(inp)

h_obj_xyz = Dense(20, activation='tanh')(h)
obj_xyz_out = Dense(3, activation='linear', name='obj_xyz_out')(h_obj_xyz)

h_obj_rot = Dense(20, activation='tanh')(h)
obj_rot_out = Dense(3, activation='linear', name='obj_rot_out')(h_obj_rot)

h_color = Dense(20, activation='tanh')(h)
color_out = Dense(3, activation='linear', name='color_out')(h_color)

h_joints = Dense(20, activation='tanh')(h)
joints_out = Dense(7, activation='linear', name='joints_out')(h_joints)

h_eff_xyz = Dense(20, activation='tanh')(h)
eff_xyz_out = Dense(3, activation='linear', name='eff_xyz_out')(h_eff_xyz)

h_eff_rot = Dense(20, activation='tanh')(h)
eff_rot_out = Dense(3, activation='linear', name='eff_rot_out')(h_eff_rot)

h_magnet = Dense(20, activation='tanh')(h)
magnet_out = Dense(1, activation='linear', name='magnet_out')(h_magnet)

fwd_model = Model(
    inputs=inp,
    outputs=[
        obj_xyz_out, obj_rot_out, color_out,
        joints_out, eff_xyz_out, eff_rot_out, magnet_out
    ],
    name='fwd_model'
)

fwd_model.summary()

In [ ]:
# from IPython.display import Image
from keras.utils import plot_model

plot_model(
    fwd_model,
    to_file='plots/kuka/magnetic/kuka_fwd_model.png',
    show_shapes=True,
    show_layer_names=True,
    show_layer_activations=True,
    dpi=300
)
# Image('plots/kuka/kuka_fwd_model.png')

In [ ]:
from keras.optimizers import Adam
from keras.losses import MeanSquaredError, MeanAbsoluteError
from keras.callbacks import TensorBoard

fwd_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={
        'obj_xyz_out': MeanSquaredError(),
        'obj_rot_out': MeanSquaredError(),
        'color_out': MeanSquaredError(),

        'joints_out': MeanSquaredError(),
        'eff_xyz_out': MeanSquaredError(),
        'eff_rot_out': MeanSquaredError(),
        'magnet_out': MeanSquaredError()
    },
    metrics={
        'obj_xyz_out': [MeanAbsoluteError()],
        'obj_rot_out': [MeanAbsoluteError()],
        'color_out': [MeanAbsoluteError()],

        'joints_out': [MeanAbsoluteError()],
        'eff_xyz_out': [MeanAbsoluteError()],
        'eff_rot_out': [MeanAbsoluteError()],
        'magnet_out': [MeanAbsoluteError()]
    }
)

fwd_history = fwd_model.fit(
    x=X_fwd_axis,
    y={
        'obj_xyz_out': y_fwd_obj_xyz,
        'obj_rot_out': y_fwd_obj_rot_axis,
        'color_out': y_fwd_color,

        'joints_out': y_fwd_joints,
        'eff_xyz_out': y_fwd_eff_xyz,
        'eff_rot_out': y_fwd_eff_rot_axis,
        'magnet_out': y_fwd_magnet
    },
    batch_size=512,
    epochs=60,
    callbacks=[TensorBoard(write_graph=False)],
    validation_split=0.2,
    shuffle=True
)

fwd_model.save('models/magnetic/kuka_fwd_60ep.keras')

In [ ]:
from keras.layers import Input, Dense
from keras.models import Sequential, Model

# inv_model = Sequential(
#     layers=[
#         Input(shape=(X_inv_reduced.shape[1],)),
#         Dense(60, activation='tanh'),
#         Dense(80, activation='tanh'),
#         Dense(80, activation='tanh'),
#         Dense(60, activation='tanh'),
#         Dense(8, activation='linear')
#     ],
#     name='inv_model'
# )

inp = Input(shape=(X_inv_reduced.shape[1],))
h = Dense(60, activation='tanh')(inp)
h = Dense(80, activation='tanh')(h)
h = Dense(80, activation='tanh')(h)
h = Dense(60, activation='tanh')(h)

joints_out = Dense(7, activation='linear', name='joints_out')(h)
magnet_out = Dense(1, activation='linear', name='magnet_out')(h)

inv_model = Model(
    inputs=inp,
    outputs=[joints_out, magnet_out],
    name='inv_model'
)

inv_model.summary()

In [ ]:
from keras.utils import plot_model

plot_model(
    inv_model,
    to_file=f'plots/kuka/magnetic/kuka_inv_model_sep.png',
    show_shapes=True,
    show_layer_names=True,
    show_layer_activations=True,
    expand_nested=True,
    dpi=300
)

In [ ]:
from keras.optimizers import Adam, AdamW
from keras.losses import MeanSquaredError, MeanAbsoluteError
from keras.callbacks import TensorBoard

inv_model.compile(
    optimizer=AdamW(learning_rate=0.001, weight_decay=0.004),
    loss={
        'joints_out': MeanSquaredError(),
        'magnet_out': MeanSquaredError()
    },
    metrics={
        'joints_out': [MeanAbsoluteError()],
        'magnet_out': [MeanAbsoluteError()]
    }
)

inv_history = inv_model.fit(
    x=X_inv_reduced,
    y={
        'joints_out': y_inv_joints,
        'magnet_out': y_inv_magnet
    },
    batch_size=512,
    epochs=100,
    callbacks=[TensorBoard(write_graph=False)],
    validation_split=0.2,
    shuffle=True
)

inv_model.save('models/magnetic/kuka_inv_sep.keras')